In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import uibcdf_tools as uibcdf_tools
import numpy as np
from simtk import unit as unit
import simtk.openmm as mm
import simtk.openmm.app as app
from mdtraj.reporters import HDF5Reporter
from sys import stdout

# Langevin dynamics with implicit solvent

In [3]:
molecular_system = msm.convert('BH3.pdb', to_form='openmm.Modeller')

In [4]:
forcefield = app.ForceField('amber99sb.xml', 'amber99_obc.xml')

In [5]:
topology = molecular_system.topology
positions = molecular_system.positions

In [6]:
system = forcefield.createSystem(topology, implicitSolvent=app.OBC2,
                                 soluteDielectric=1.0, solventDielectric=78.5)
# Ver http://docs.openmm.org/7.1.0/userguide/application.html#amber-implicit-solvent
# Puedo tambien poner implicitSolventKappa=1.0/nanometer

In [7]:
temperature = 325*unit.kelvin
integration_timestep = 2.0*unit.femtosecond
damping = 1.0/unit.picosecond

integrator = mm.LangevinIntegrator(temperature, damping, integration_timestep)

In [8]:
platform = mm.Platform.getPlatformByName('CUDA')

In [9]:
properties = {'CudaPrecision': 'mixed'}

In [10]:
simulation = app.Simulation(topology, system, integrator, platform, properties)

In [11]:
simulation.context.setPositions(positions)

print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
simulation.minimizeEnergy()
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

7482.029606661797 kJ/mol
-3603.6824523577893 kJ/mol


In [12]:
simulation.context.setVelocitiesToTemperature(temperature)

In [13]:
md_time = 10.0 * unit.nanoseconds
saving_time_interval = 10.0 * unit.picoseconds
log_time_interval = 0.2 * unit.nanoseconds

md_steps = np.rint(md_time/integration_timestep).astype(int)
saving_steps_interval = np.rint(saving_time_interval/integration_timestep).astype(int)
log_steps_interval = np.rint(log_time_interval/integration_timestep).astype(int)

In [14]:
reporter_log_stdout = app.StateDataReporter(stdout, log_steps_interval, step=True, time=True,
                                            totalEnergy=True, temperature=True,
                                            progress=True, remainingTime=True,
                                            speed=True, totalSteps=md_steps, separator='\t')


reporter_logfile = app.StateDataReporter('traj.log', log_steps_interval, step=True, time=True,
                                            potentialEnergy=True, kineticEnergy=True,
                                            totalEnergy=True, temperature=True,
                                            progress=True, remainingTime=True,
                                            speed=True, totalSteps=md_steps, separator='\t')

reporter_h5file = HDF5Reporter('traj.h5', saving_steps_interval, coordinates=True, time=True,
                               potentialEnergy=True, kineticEnergy=True, temperature=True)

simulation.reporters.append(reporter_log_stdout)
simulation.reporters.append(reporter_logfile)
simulation.reporters.append(reporter_h5file)

In [15]:
simulation.step(md_steps)

#"Progress (%)"	"Step"	"Time (ps)"	"Total Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
2.0%	100000	200.00000000022686	-920.837398393162	328.7657925698578	0	--
4.0%	200000	400.00000000118183	-956.7589533639177	330.9325818739608	1.61e+03	8:34
6.0%	300000	599.9999999996356	-1052.5232784055584	322.07748622162006	1.61e+03	8:23
8.0%	400000	799.9999999949063	-1168.620558288614	296.03706293092904	1.61e+03	8:14
10.0%	500000	999.9999999901769	-1120.9779447305384	332.29361505453494	1.61e+03	8:03
12.0%	600000	1199.9999999854476	-1242.614510404725	315.11330828622823	1.61e+03	7:52
14.0%	700000	1399.9999999807183	-1119.054638759907	328.12751249015355	1.61e+03	7:41
16.0%	800000	1599.9999999759889	-1282.1802287940004	294.4264196367732	1.61e+03	7:30
18.0%	900000	1799.9999999712595	-1245.8426558561341	298.430162516121	1.61e+03	7:20
20.0%	1000000	1999.9999999665301	-1121.9950851377455	344.9395834936983	1.61e+03	7:09
22.0%	1100000	2199.9999999618008	-1116.450888776495	347.747720834

In [16]:
reporter_h5file.close()

In [17]:
msm.view('traj.h5')

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/tables/leaf.py:544: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  key = numpy.array(key)


NGLWidget(max_frame=999)